This is a demonstration of the features of IJuliaTimeMachine.

# What IJulia Provides

IJulia already provides some historical information. `In` records the inputs to cells, and `Out` records their outputs.  `IJulia.n` is the number of the current cell.

In [1]:
1+1

2

In [2]:
In[1]

"1+1"

In [3]:
Out[1]

2

# Setting up IJuliaTimeMachine

I recommend assigning `IJuliaTimeMachine` a shorter name, like `TM` as below.

In [4]:
import IJuliaTimeMachine
TM = IJuliaTimeMachine

┌ Info: Precompiling IJuliaTimeMachine [c6113210-d3d9-4301-930e-a2f8bca719bb]
└ @ Base loading.jl:1278


IJuliaTimeMachine

If you use it a lot, and don't want to keep typing the `TM` prefix, you can instead type `using IJuliaTimeMachine`.
This will import `@past` and `vars`.

# Recalling past states

In [5]:
x = 1+1

2

In [6]:
x = randn(3)

3-element Array{Float64,1}:
  0.3278969893228005
 -1.4969328684500842
  1.1073820854652396

Say that I just accidentally change the value of the variable `x`, and I'd like to know what it's value was after cell 5.  I can recover the state of variables from then with `@past`.

In [7]:
TM.@past 5
x

2

The value of `ans` was also recalled.

In [8]:
ans

2

The Time Machine stores a `deepcopy` of every variable.  While this is inefficient, it allows us to recover a vector, even if some other cell changes one of its entries.

In [9]:
vec = collect(1:4)

4-element Array{Int64,1}:
 1
 2
 3
 4

In [10]:
vec[3] = 100
vec

4-element Array{Int64,1}:
   1
   2
 100
   4

In [57]:
TM.@past 9
vec

4-element Array{Int64,1}:
 1
 2
 3
 4

By default `@past` also recalls the output of the past cell, and so that output appears in the display.

In [58]:
TM.@past 9

4-element Array{Int64,1}:
 1
 2
 3
 4

This can be very useful because IJulia's Out stores a pointer to an array, rather than the array.
This means that the value recalled can be changed, like this.

In [13]:
y = [1;2;3]

3-element Array{Int64,1}:
 1
 2
 3

In [14]:
Out[13]

3-element Array{Int64,1}:
 1
 2
 3

In [15]:
y[3] = 0
Out[13]

3-element Array{Int64,1}:
 1
 2
 0

Note that the last element changed to a 0. This does not happen with the values stored by the Time Machine.

In [16]:
TM.@past 13

3-element Array{Int64,1}:
 1
 2
 3

The Time Machine only stores variables it can effectively copy.
Right now, it can not copy functions.  

In [17]:
cell = IJulia.n
f(x) = x
f(1)

1

In [18]:
y = 2
f(x) = 2x
f(1)

2

In [19]:
TM.@past cell

1

In [20]:
f(1)

2

But, if the only thing that changes in a function is a global variable, then you can essentially recover the function.

In [21]:
f(x) = y*x

f (generic function with 1 method)

In [22]:
cell = IJulia.n
y = 3
f(1)

3

In [23]:
y = 4
f(1)

4

In [24]:
TM.@past cell

3

In [25]:
f(1)

3

You can stop the Time Machine from saving.

In [26]:
TM.saving!(false)

save_state (generic function with 1 method)

In [27]:
z = "hello!"

"hello!"

In [28]:
TM.@past 27

LoadError: State 27 was not saved.

And, you can make it start saving again.

In [29]:
TM.saving!(true)

1-element Array{Function,1}:
 save_state (generic function with 1 method)

In [30]:
z = "hi :)"

"hi :)"

In [31]:
z = "overwrite that"

"overwrite that"

In [32]:
TM.@past 30
z

"hi :)"

If we really want to forget the past, or just save memory, we can clear some history.  Just give a list of the cells to be cleared.  If no list is specified, it clears all of them. This is the safest option, because to free up the memory used by a variable, one must clear *every* cell in which that variable exists.

In [33]:
TM.clear_past([30])

In [34]:
TM.@past 30

LoadError: State 30 was not saved.

To see what can go wrong with a `clear_past` call, let's try getting rid of the array `[1;2;3]` which was bound to `y` in cell 13.

In [35]:
TM.clear_past([13])

We can get a dictionary of the variables that are stored for any cell by using the `vars` function. If we generate this list for any cell after 13, we will see that `y` is still there.

In [36]:
TM.vars(20)

Dict{Any,Any} with 4 entries:
  :vec  => [1, 2, 3, 4]
  :y    => [1, 2, 3]
  :cell => 17
  :x    => 2

If you want to see exactly what was stored, or exactly what variables are being overwritten, look at `TM.VX.past`.

In [35]:
TM.VX.past[23].vars

Dict{Any,Any} with 4 entries:
  :x    => 0x5a94851fb48a6e05
  :y    => 0x867b4bb4c42e5661
  :cell => 0x6fd433390b9283dd
  :vec  => 0x8c7dfaa1a09ee6fa

In [36]:
TM.VX.past[23].ans

4

The big table of all the stored variables is `TM.VX.store`

In [37]:
TM.VX.store

Dict{Any,Any} with 13 entries:
  0xc70ed234e0a3ad59 => 17
  0xf4e13e0713120297 => "hi :)"
  0x3688cf5d48899fa6 => 3
  0x5a94851fb48a6e05 => 2
  0x867b4bb4c42e5661 => 4
  0x0296034ccc0d24cb => [1, 2, 0]
  0x74350a76d34418a6 => [1, 2, 100, 4]
  0x6fd433390b9283dd => 22
  0x8c7dfaa1a09ee6fa => [1, 2, 3, 4]
  0xd22721a98cab7f9d => [1, 2, 3]
  0xb98a8ec9a0959679 => "hello!"
  0x700bc5643854405f => "overwrite that"
  0xe9a36d00ccdaa34c => [0.327897, -1.49693, 1.10738]

These are stored by hash so that each piece of data is only stored once. This reduces wasted memory.

But, if you have some big piece of data that you do not want stored, you can tell the Time Machine not to do that.
The function `dontsave_val` prevents the TM from saving any variable whose value equals the value of the variable given.

In [38]:
BIG = randn(1000,1000)
TM.dontsave_val(BIG)

In [39]:
TM.vars(38)

Dict{Any,Any} with 5 entries:
  :vec  => [1, 2, 3, 4]
  :y    => 3
  :cell => 22
  :z    => "hi :)"
  :x    => 2

In [55]:
cell = IJulia.n
X = 42
TM.dontsave_val(X)

In [56]:
TM.vars(cell)

Dict{Any,Any} with 6 entries:
  :vec  => [1, 2, 3, 4]
  :y    => 3
  :cell => 55
  :zzz  => 1
  :z    => "hi :)"
  :x    => 2

If we change the value of X, it will now be saved. If we change an entry of BIG this will not appear to be a change, because the hash of BIG records where the variable points, not the data 

In [62]:
@trace TM.can_copy(Int64)

LoadError: LoadError: UndefVarError: @trace not defined
in expression starting at In[62]:1

In [41]:
TM.vars(40)

Dict{Any,Any} with 5 entries:
  :vec  => [1, 2, 3, 4]
  :y    => 3
  :cell => 22
  :z    => "hi :)"
  :x    => 2

In [45]:
hash(TM.VX)

0x82fb1301161f84e3

In [46]:
zzz = 1

1

In [47]:
hash(TM.VX)

0x82fb1301161f84e3

In [48]:
big = randn(1000,1000)
TM.dontsave_var(big)

In [51]:
TM.vars(48)

Dict{Any,Any} with 6 entries:
  :vec  => [1, 2, 3, 4]
  :y    => 3
  :cell => 22
  :zzz  => 1
  :z    => "hi :)"
  :x    => 2

In [52]:
hash(x)

0x5a94851fb48a6e05

In [53]:
hash(2)

0x5a94851fb48a6e05

# Running big jobs in threads

The other feature of Time Machine is that it lets you run intensive jobs in threads, so that you can get other work done while they are running.  If you have a multicore machine, you can also view this as a way to manage running a bunch of experiments from Jupyter.  The key is to wrap the jobs in `TM.@thread begin`, followed by `end`.  Jobs that are running inside a `@thread` block work on sandboxed variables. They start by copying all variables the exist when they are called. But, they do not change the values of any variables. To access the values of the variables they change, use `@past`.

When the jobs finish, their result is stored in `Out`, and you can access their state from `past`.
Unfortunately, if the jobs contain any print statements, they can show up in other cells.

To see which jobs are running, look at `TM.running`.  `TM.finished` contains a list of those that have finished.

In the examples below, I will simulate the delay of a long-running job with `sleep`.  As you will see, results will change after jobs finish.

In [37]:
x = collect(1:3)
y = 1

1

In [38]:
t0 = time()
n = IJulia.n
TM.@thread begin
    y = y + 1
    push!(x,y)
    sleep(5)
    sum(y)
end

Task (runnable) @0x0000000113cad450

In [39]:
println("After $(time() - t0) seconds.")
Out[n]

After 0.42882800102233887 seconds.


Task (runnable) @0x0000000113cad450

In [40]:
sleep(10)

In [41]:
println("After $(time() - t0) seconds.")
Out[n]

After 10.49637222290039 seconds.


2

In [42]:
x, y

([1, 2, 3], 1)

In [43]:
TM.@past n
x, y

([1, 2, 3, 2], 2)

There is a subtle reason that I put the `sleep(10)` statement on a separate line.
The output of finished jobs is only inserted into `Out` at the start of the execution of the first cell that is run after the job finishes. So, it is possible to go one cell without the output being correct.
If you want to test it, put the `sleep(10)`  inside the next cell, and then run the cells in quick succession.

You can not put two `@thread` statements into one cell.

In [44]:
n = IJulia.n
TM.@thread begin
    x = x + 1
    y[1] = 3
    sum(y)
end
TM.@thread begin
    x = x + 1
    y[1] = 4
    sum(y)
end

@thread can be called at most once per cell.


But, you can have many running at once.  That's the point!

In [45]:
function intense(n)
    sleep(n)
    println("I slept for $(n) seconds!")
    n^2
end

intense (generic function with 1 method)

In [46]:
t0 = time()
n1 = IJulia.n
TM.@thread intense(10)

Task (runnable) @0x0000000113caf3d0

In [47]:
println("After $(time()-t0) seconds")
n2 = IJulia.n
TM.@thread intense(11)

After 0.0682060718536377 seconds


Task (runnable) @0x0000000114a20d90

In [48]:
println("After $(time()-t0) seconds")
n3 = IJulia.n
TM.@thread intense(2)

After 0.127471923828125 seconds


Task (runnable) @0x0000000115412890

In [49]:
println("After $(time()-t0) seconds")
TM.running

After 0.1977980136871338 seconds


Set{Any} with 3 elements:
  47
  48
  46

In [50]:
sleep(3)

I slept for 2 seconds!


In [51]:
println("After $(time()-t0) seconds")
TM.running

After 3.899488925933838 seconds


Set{Any} with 2 elements:
  47
  46

In [52]:
println("After $(time()-t0) seconds")
TM.finished

After 3.901526927947998 seconds


2-element Array{Int64,1}:
 38
 48

In [53]:
sleep(10)
println("After $(time()-t0) seconds")
TM.running

I slept for 10 seconds!
I slept for 11 seconds!
After 13.90711498260498 seconds


Set{Any}()

In [54]:
Out[n1], Out[n2], Out[n3]

(100, 121, 4)

You can also get notifications of when jobs finish.
These can be sent to Jupyter, in which case they will just print.
You can also send them to the terminal in which Jupyter is running, or both.

In [55]:
TM.notify_jupyter!(true)
TM.notify_terminal!(true)
@TM.thread begin
    println("this will run for 3 seconds")
    sleep(3)
end

this will run for 3 seconds


Task (runnable) @0x0000000113caead0

Cell 55 finished.
